The steps of the code are as follows:
1. Import the RTKLIB corrected Rinex drone data. 
2. Import the Timestamp.MRK file from the UAV.
3. Calculate the camera specific positions from these two files.
4. Scrape the EXIF-data from the original photos to append the pitch, roll, and yaw to the location file. 
    This data can then be used to import into Agisoft.

In [12]:
# Import numerical tools
import numpy as np
# Import pyplot for plotting
import matplotlib.pyplot as plt
#Import pandas for reading in and managing data
import pandas as pd
import math
# Magic function to make matplotlib inlineSet the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.; other style specs must come AFTER
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
#Comment the above line and uncomment the line below if svg graphics are not working in your browser.
#%config InlineBackend.figure_formats = {'png', 'retina'}
import os
import glob
import os.path
import re

Set the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.

In [2]:
# Set the filename. Make sure to keep this value in parenthesis, so that the program reads in this filename 
# number as a named file, and not as an integer.
filename = '101_0340'
# Set the date of the flight to search for the working directory for each time this code is run.
date = '20220505'

In [3]:
# Set your working directory
# Hourglass processing work should be in the folder 'HG_2022_PPK_Processing' and should be parsed out by date
# Since each date will have its own folder, the date in the working directory should be the only thing changed in
# this form.
os.chdir('/Users/f67f911/Desktop/HG_2022_PPK_Processing/' + date + '/Raw_Files/')

Step 1: Read in the already corrected RINEX file from the drone. This file should always be saved in your directory as filename_Rinex.csv so that the following code works. This is from the drone capturing positional location throughout the entirety of the flight, so there will be many more rows of data than there are pictures taken during that flight. 

In [4]:
# Since we have set our working directory, and we have specified the file name, we just need to search for the
# file that is followed by Rinex and is a .csv
RTKLIB_record = pd.read_csv(filename + '_Rinex.csv')
# View the head of the data to make sure it has been read in correctly
RTKLIB_record.head()
# Uncomment the following code if you would like to view the length of the file. 
# len(RTKLIB_record)

,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio
0,2208,399470.8,45.83509,-110.932688,2314.4909,1,6,0.0149,0.0066,0.0224,0.0046,0.0076,0.0148,-4.2,527.1
1,2208,399471.0,45.83509,-110.932688,2314.4880,1,6,0.0147,0.0065,0.0220,0.0045,0.0075,0.0145,-4.0,526.5
2,2208,399471.2,45.83509,-110.932688,2314.4906,1,6,0.0144,0.0064,0.0217,0.0044,0.0073,0.0143,-3.8,526.0
3,2208,399471.4,45.83509,-110.932688,2314.4864,1,6,0.0142,0.0062,0.0213,0.0043,0.0072,0.0141,-3.6,525.8
4,2208,399471.6,45.83509,-110.932688,2314.4932,1,6,0.0139,0.0061,0.0210,0.0042,0.0070,0.0138,-3.4,525.7


Step 2: Read in the timestamp data as is from the UAV. 

In [32]:
# Again, since we have a single working directory, read in the timestamp data with the name of the file.
timestamp_record = pd.read_table(filename + '_Timestamp.MRK', header = None)
# The UAV used, a P4 RTK, always collects timestamp files with the data in the same order. Therefore, we can set the 
# column names for the timestamp file read in. 
timestamp_record.columns = ['Photo', 'GPS_Date','% GPST','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m','std_North_m, std_East_m, std_Ele_m','RTK_status_flag']
# View the head of the data to make sure it has been read in correctly. 
timestamp_record.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag
0,1,399477.727242,[2208],"-9,N","-4,E","194,V","45.83506059,Lat","-110.93269097,Lon","2317.334,Ellh","0.012661, 0.021600, 0.029364","50,Q"
1,2,399481.901610,[2208],"10,N","-16,E","193,V","45.83513023,Lat","-110.93275864,Lon","2307.852,Ellh","0.012964, 0.022205, 0.029983","50,Q"
2,3,399485.464902,[2208],"15,N","-21,E","193,V","45.83519589,Lat","-110.93282491,Lon","2298.959,Ellh","0.012870, 0.021786, 0.030213","50,Q"
3,4,399486.986303,[2208],"16,N","-21,E","192,V","45.83520890,Lat","-110.93283758,Lon","2297.254,Ellh","0.013342, 0.022087, 0.030188","50,Q"
4,5,399491.646347,[2208],"0,N","3,E","194,V","45.83522713,Lat","-110.93285622,Lon","2309.734,Ellh","0.013127, 0.021502, 0.030736","50,Q"


In [6]:
timestamp = timestamp_record

Note status flag values - 0: no positioning; 16: single-point positioning mode; 34:RTK floating solution; 50: RTK fixed solution. When flag of a photo is not equal to 50, it is recommended that you should not use that image in further processing.

Clean the timestamp file to convert non-numeric text in columns to numeric

When looking at our columns, we can see that there are many numbers followed by letters. We need to get rid of those numbers in order to continue with our analysis. This code can definitely be cleaned up. For example, right now it works because all of the flight logs I am using take place in the United States, meaning that the letters following the timestamp data will all be the same. However, this is not necessarily the case in other areas.

In the future, I could try to remove anything from the columns that is not an integer, so that letters and commas are removed. However, right now, this works fine. 

In [ ]:
timestamp.dtypes

In [ ]:
# Convert the object columns to strings
timestamp[['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m',
           'std_North_m, std_East_m, std_Ele_m','RTK_status_flag']] = timestamp[['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m',
           'std_North_m, std_East_m, std_Ele_m','RTK_status_flag']].astype(string)
timestamp.dtypes

In [ ]:
timestamp['Northing_diff_mm'] = timestamp['Northing_diff_mm'].str.replace(r"[a-zA-Z,]",'')
timestamp

In [7]:
str_col = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m',
           'std_North_m, std_East_m, std_Ele_m','RTK_status_flag']

In [25]:
test = re.findall(r'-?\d+', timestamp['Northing_diff_mm'])

TypeError: expected string or bytes-like object

In [34]:
# Create a for loop that cycles through the values in each column of the dataframe and 
# gets rid of values that are not numeric. 
# This step is necessary for further processing
for col in timestamp:
    for name in str_col:
        if col == name:
            
            timestamp[col] = timestamp[col].str.extract("(-\d+)", expand = False)
# Check the dataframe to make sure that it worked
timestamp

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag
0,1,399477.727242,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,399481.901610,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,399485.464902,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,399486.986303,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,399491.646347,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
213,214,400209.409687,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,215,400210.931076,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,216,400214.819526,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,217,400218.353494,[2208],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
